In [ ]:
import cv2
import numpy as np
import time
import urllib.request
import os


wiper_thresh = 40000
kernel = np.ones((5,5),np.uint8)

canvas = None
clear = False

noiseth = 100

cx_prev = 0
cy_prev = 0

address = "http://192.168.29.37:8080/shot.jpg?rnd=642465"
folder_path = "path" 

i = 0

while True :
    
    image_resp = urllib.request.urlopen(address)
    image_np = np.array(bytearray(image_resp.read()), dtype = np.uint8)
    frame = cv2.imdecode(image_np, -1)
    (h, w, c) = frame.shape
    frame = cv2.resize(frame, (w//2, h//2))
    frame = cv2.flip(frame, 1)
    cv2.imshow("video", frame)
    frame = cv2.flip( frame, 1 )
    if canvas is None:
        canvas = np.zeros_like(frame)
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    lower_range  = np.array([23,155,140])
    upper_range = np.array([54,255,255])
    mask = cv2.inRange(hsv, lower_range, upper_range)
    
    #ret, mask = cv2.threshold(frame, 225, 255, cv2.THRESH_BINARY)
    
    mask = cv2.erode(mask,kernel,iterations = 1)
    mask = cv2.dilate(mask,kernel,iterations = 2)

    contours, hierarchy = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if contours and cv2.contourArea(max(contours, key = cv2.contourArea)) > noiseth:
                
        c = max(contours, key = cv2.contourArea)
        area = cv2.contourArea(c)
        x2,y2,w,h = cv2.boundingRect(c)
        cx = x2 + w//2
        cy = y2 + h//2
        if cx_prev == 0 and cy_prev == 0:
            cx_prev, cy_prev = cx, cy
            
        else:
            canvas = cv2.line(canvas, (cx,cy),(cx_prev,cy_prev), (255,0,0), 4)
        cx_prev = cx
        cy_prev = cy
        if area > wiper_thresh:
           cv2.putText(canvas,'Clearing Canvas', (100,200), 
           cv2.FONT_HERSHEY_SIMPLEX,2, (0,0,255), 5, cv2.LINE_AA)
           clear = True 

    else:

        cx_prev,cy_prev =0,0
    
    frame = cv2.add(frame,canvas)
    stacked = np.hstack((canvas,frame))
    cv2.imshow('PAGE',cv2.resize(stacked,None,fx=0.6,fy=0.6))

    if clear == True:
        time.sleep(3)
        canvas = None
        clear = False
        x1, y1 = 0, 0
    k = cv2.waitKey(3) & 0xFF
    if k == 27:
        break
    if k == ord('c'):
        canvas = None
    
    #if k == ord('s'):
        
        #Image_save = cv2.resize(canvas, None, fx=0.6, fy=0.6)
        #os.chdir(folder_path)
        #cv2.imwrite('img_' + str(i) +'.jpg', Image_save)
        #i += 1

    cv2.waitKey(1)
cv2.destroyAllWindows()